In [1]:
import sys
import os
sys.path.append(os.path.join(sys.path[0].split('BecaNLP')[0],'BecaNLP/Utils'))

import torch
import torch.nn as nn
import NLPUtils as nlp

%load_ext autoreload
%autoreload 2

In [8]:
corpus, vocab = nlp.datasets.utils.get_toy_corpus(10)

In [8]:
x = [torch.tensor([1,2,0,5,2,3]).view(-1,1),
     torch.tensor([1,4,4,3]).view(-1,1),
     torch.tensor([1,2,3]).view(-1,1)]
x_padded = nn.utils.rnn.pad_sequence(x)
x_packed = nn.utils.rnn.pack_padded_sequence(x_padded,lengths=[6,4,3])
x_packed.data

tensor([[1],
        [1],
        [1],
        [2],
        [4],
        [2],
        [0],
        [4],
        [3],
        [5],
        [3],
        [2],
        [3]])

In [114]:
def rnn_collate_fn(data_batch):
    """
    Función para pasarle al DataLoader cuando entrenamos una RNN.
    """
    x_batch, y_batch = zip(*data_batch)
    lenghts = torch.tensor([sample.size(0) for sample in x_batch])
    sorted_lenghts_idx = torch.argsort(lenghts,descending=True)
    lenghts = lenghts[sorted_lenghts_idx]
    x_batch = torch.nn.utils.rnn.pad_sequence(x_batch, batch_first=True)
    x_batch = x_batch[sorted_lenghts_idx]
    x_batch = torch.nn.utils.rnn.pack_padded_sequence(x_batch, batch_first=True, lengths=lenghts)
    return x_batch, torch.tensor(y_batch)

N = 512
max_len = 100
np.random.seed(623423784)
#idx = np.random.permutation(N)
lenghts = np.random.randint(10,max_len,N)
x = [torch.randint(10,(l,)) for l in lenghts]
#print(collate_fn())

class dummy_dataset(torch.utils.data.Dataset):
    
    def __init__(self,N):
        max_len = 100
        n_features = 3
        lenghts = np.random.randint(10,max_len,N)
        self.x = [torch.randint(10,(l,n_features)).type(torch.float) for l in lenghts]
        self.y = torch.randint(10,(N,))
        
    def __getitem__(self,idx):
        return self.x[idx], self.y[idx]
    
    def __len__(self):
        return len(self.y)
    
data = dummy_dataset(1000)